In [1]:
# ATTENZIONE: per eseguire gli esempi relativi al calcolo parallelo in questa pagina 
# ci si deve spostare nell'environment "parallel" (anzichè "root")

In [2]:
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)

from time import sleep

def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b


In [3]:
future = e.submit(slowadd, 1, 2)
future

<Future at 0x2850e10d8b0 state=running>

In [4]:
future.result()

3

In [7]:
%%time
results = [slowadd(i, i, delay=1) for i in range(10)]

CPU times: total: 0 ns
Wall time: 10 s


In [8]:
%%time
futures = [e.submit(slowadd, 1, 1, delay=1) for i in range(10)]
results = [f.result() for f in futures]

CPU times: total: 0 ns
Wall time: 3.03 s


In [9]:
import loky
from loky import ProcessPoolExecutor
e = ProcessPoolExecutor(4)

In [10]:
from time import sleep
def slowadd(a, b, delay=1):
    sleep(delay)
    return a + b

In [11]:
# Notare che il CPU time non scorre durante il secondo di sleep; quindi dobbiamo misurare 
# il wall time

In [12]:
future = e.submit(slowadd, 1, 2)

In [15]:
future

<Future at 0x2850e087eb0 state=finished returned int>

In [16]:
future.result()

3

In [18]:
%%time
results = [slowadd(i, i, delay=1) for i in range(10)]

CPU times: total: 0 ns
Wall time: 10.1 s


In [19]:
%%time
futures = [e.submit(slowadd, 1, 1, delay=1) for i in range(10)]
results = [f.result() for f in futures]

CPU times: total: 0 ns
Wall time: 3.03 s


In [20]:
# si ottiene uno speed-up di circa 3.3 sia con processi che con thread 

In [22]:
import pandas as pd
import numpy as np
#filename = "c:\\documenti\\demografia\\download\\database\\comuni4.csv"
#filename = "c:\\documenti\\demografia\\download\\database\\comuni4.csv"
filename = "comuni4.csv"
df = pd.read_csv(filename, header=None)
t = df.iloc[1:7902].values
t.reshape(7901,8)

array([[1, 1001, "Aglie'", ..., 13.28, 45.36472222222223,
        7.767777777777778],
       [1, 1002, 'Airasca', ..., 15.7, 44.9175, 7.491111111111111],
       [1, 1003, 'Ala di Stura', ..., 46.09, 45.31611111111111,
        7.302777777777778],
       ...,
       [20, 91085, 'Siniscola', ..., 199.96, 40.57472222222223,
        9.694166666666668],
       [20, 91086, 'Sorgono', ..., 56.19, 40.026944444444446, 9.105],
       [20, 91087, 'Suni', ..., 47.32, 40.281666666666666,
        8.54861111111111]], dtype=object)

In [24]:
def distanza(a,b):
    import math as mt
    gradlat = [110.869, 110.887, 110.904, 110.922, 110.941, 110.959, 110.978, 110.996, 111.015, 111.035, 111.54, 111.073, 111.093, 111.112, 111.132, 111.151, 111.171, 111.190, 111.210, 111.229]
    gradlon = [95.504, 94.493 ,93.453, 92.385, 91.288, 90.164, 89.012, 87.832, 86.626, 85.394, 84.135, 82.851, 81.541, 80.206, 78.847, 77.463, 76.056, 74.625, 73.172, 71.696]
    dlat=float(a[6])-float(b[6])
    dlon=float(a[7])-float(b[7])
    medlat=(float(a[6])+float(b[6]))/2
    kmlat=dlat*gradlat[round(medlat)-31]
    kmlon=dlon*gradlon[round(medlat)-31]
    dist=mt.sqrt(kmlat**2+kmlon**2)
    return dist

In [25]:
def pop20km(t):
    p20 = []
    for i in range(790):
        pop20 = 0
        for j in range(7901):
            if ( distanza(t[i],t[j]) < 20 ):
                pop20 = pop20 + t[j][4]
        p20.append(pop20)
#        if ( pop20 > 5e5 ):
#            print(t[i][2]+" "+str(pop20))
    return p20

In [26]:
%load_ext snakeviz

In [27]:
%%time 
## tempo medio di esecuzione = fino a 6/7 min ca (dipende dall'HW), esecuzione sequenziale
results = [pop20km(t) for i in range(10)]

CPU times: total: 3min 55s
Wall time: 4min 15s


In [29]:
from concurrent.futures import ThreadPoolExecutor
e = ThreadPoolExecutor(4)

In [30]:
%%time 
## tempo medio di esecuzione = fino a 6/7 min ca (dipende dall'HW), esecuzione con thread paralleli
futures = [e.submit(pop20km,t) for i in range(10)]
results = [f.result() for f in futures]

CPU times: total: 3min 59s
Wall time: 4min 18s


In [31]:
import loky
from loky import ProcessPoolExecutor
e = ProcessPoolExecutor(4)

In [32]:
%%time
futures = [e.submit(pop20km,t) for i in range(10)]
results = [f.result() for f in futures]

CPU times: total: 46.9 ms
Wall time: 55.6 s


In [ ]:
# Le operazioni di ingresso/uscita favoriscono la parallelizzazione con processi rispetto
# a quella con thread perchè nel secondo caso le richieste di I/O provengono da un unico 
# processo e sono gestite in maniera sequenziale; nel caso multi-processo, invece, ognuno 
# dei processi paralleli fa le sue richieste in modo autonomo e questo porta ad un migliore 
# grado di parallelizzazione (le istruzioni di I/O sono quelle dominanti per durata)